In [60]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import Ridge

data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df=df[df.index>'2017-05-30']

train=df[df.index<'2017-09-01']

features = ['TBondsOpenValue', 'UnemploymentValue', 'BTCOpen', 'BTCOpenTM1', 'BTCOpenTM2',\
        'BTCOpenTM3', 'BTCOpenTM4', 'BTCOpenTM5', 'BTCOpenTM6', 'BTCOpenTM7','GDP','SandPValue',\
            'ETHOpenTM1', 'ETHOpenTM2',\
        'ETHOpenTM3', 'ETHOpenTM4', 'ETHOpenTM5', 'ETHOpenTM6', 'ETHOpenTM7']

train=train.dropna()


dtrain = xgb.DMatrix(train.loc[:, features].values, \
                     label = train.loc[:, 'ETHOpen'].values)

params = {}
params['booster']  = 'gbtree'
params['objective'] = 'reg:linear'
params['max_depth'] = 6
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['silent'] = 1
params['eval_metric'] = 'rmse'
num_round = 50
eval_list  = [(dtrain,'train')]

train['Date'] = train.index.values

print('Training xgb model:')
bst = xgb.train(params, dtrain, num_round, eval_list)

print('Trainign Ridge Regression')
lr = Ridge()
lr.fit(train.loc[:, features].values, \
       train.loc[:, 'ETHOpen'].values)

test = df[df.index>='2017-09-01']
test=test.dropna()

while True:
    dtest = xgb.DMatrix(test[features].values)
    xgb_pred = bst.predict(dtest)
    lr_pred = lr.predict(test[features].values)
    test['ETHOpen'] = 0.2*xgb_pred+0.8*lr_pred

    target = train['ETHOpen']
    
    done = 1
    
    if done:
        print("Prediction: {}".format(test['ETHOpen']))
        break



training xgb model...
[0]	train-rmse:199.135
[1]	train-rmse:144.441
[2]	train-rmse:104.401
[3]	train-rmse:75.0942
[4]	train-rmse:54.8394
[5]	train-rmse:39.909
[6]	train-rmse:29.0933
[7]	train-rmse:21.4535
[8]	train-rmse:16.1608
[9]	train-rmse:12.3654
[10]	train-rmse:9.40966
[11]	train-rmse:7.69997
[12]	train-rmse:6.03475
[13]	train-rmse:4.85785
[14]	train-rmse:3.90114
[15]	train-rmse:3.36007
[16]	train-rmse:2.67474
[17]	train-rmse:2.20594
[18]	train-rmse:1.83359
[19]	train-rmse:1.61321
[20]	train-rmse:1.35221
[21]	train-rmse:1.20292
[22]	train-rmse:1.06671
[23]	train-rmse:0.909228
[24]	train-rmse:0.798715
[25]	train-rmse:0.715271
[26]	train-rmse:0.659335
[27]	train-rmse:0.573475
[28]	train-rmse:0.498174
[29]	train-rmse:0.441628
[30]	train-rmse:0.397919
[31]	train-rmse:0.355675
[32]	train-rmse:0.309196
[33]	train-rmse:0.281217
[34]	train-rmse:0.246979
[35]	train-rmse:0.216129
[36]	train-rmse:0.19393
[37]	train-rmse:0.167735
[38]	train-rmse:0.157404
[39]	train-rmse:0.136133
[40]	train-rm

In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.linear_model import Lasso

data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df=df[df.index>'2017-05-30']

train=df[df.index<'2017-09-01']

features = ['TBondsOpenValue', 'UnemploymentValue', 'BTCOpen', 'BTCOpenTM1', 'BTCOpenTM2',\
        'BTCOpenTM3', 'BTCOpenTM4', 'BTCOpenTM5', 'BTCOpenTM6', 'BTCOpenTM7','GDP','SandPValue',\
            'ETHOpenTM1', 'ETHOpenTM2',\
        'ETHOpenTM3', 'ETHOpenTM4', 'ETHOpenTM5', 'ETHOpenTM6', 'ETHOpenTM7']
value = ['ETHOpen']

dtrain = xgb.DMatrix(train.loc[:, features].values, \
                     label = train.loc[:, 'ETHOpen'].values)


def lasso_regression(train, predictors, alpha, models_to_plot={}):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg.fit(train[predictors],train['ETHOpen'])
    y_pred = lassoreg.predict(train[predictors])
    
    #Check if a plot is to be made for the entered alpha
    if alpha in models_to_plot:
        #print(models_to_plot[alpha])
        #print(alpha)
        #print(train)
        plt.subplot(models_to_plot[alpha])
        plt.tight_layout()
        plt.plot(train[train.index],y_pred)
        plt.plot(train[train.index],train[value],'.')
        plt.title('Plot for alpha: %.3g'%alpha)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-train)**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [123]:
predictors=train.loc[:, features].values
#predictors.extend(['x_%d'%i for i in range(2,20)])

#Define the alpha values to test
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

#Initialize the dataframe to store coefficients
col = ['rss','intercept'] + features
ind = ['alpha_%.2g'%alpha_lasso[i] for i in range(0,10)]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

#Define the models to plot
models_to_plot = {1e-10:231, 1e-5:232,1e-4:233, 1e-3:234, 1e-2:235, 1:236}

#Iterate over the 10 alpha values:
for i in range(10):
    coef_matrix_lasso.iloc[i,] = lasso_regression(dtrain,predictors , alpha_lasso[i], models_to_plot)


TypeError: only integer scalar arrays can be converted to a scalar index

In [58]:
coef_matrix_lasso.apply(lambda x: sum(x.values==0),axis=1)

alpha_1e-15      1
alpha_1e-10      1
alpha_1e-08      1
alpha_1e-05      1
alpha_0.0001     9
alpha_0.001     17
alpha_0.01      18
alpha_1         18
alpha_5         18
alpha_10        19
dtype: int64

In [116]:
lassoreg = Lasso(alpha=1e-10,normalize=True, max_iter=1e5)
lassoreg.fit(train[predictors.values],train.values)

KeyError: '[ (154.6875, 4.2999999999999998, 228.58000000000001, 228.58000000000001, 228.63999999999999, 228.63999999999999, 220.69999999999999, 244.96000000000001, 247.75, 264.25999999999999, 18569.099999999999, 2411.0, 228.58000000000001, 228.63999999999999, 228.63999999999999, 220.69999999999999, 244.96000000000001, 247.75, 264.25999999999999)\n (154.6875, 4.2999999999999998, 228.58000000000001, 228.63999999999999, 228.63999999999999, 220.69999999999999, 244.96000000000001, 247.75, 264.25999999999999, 255.77000000000001, 18569.099999999999, 2411.0, 228.63999999999999, 228.63999999999999, 220.69999999999999, 244.96000000000001, 247.75, 264.25999999999999, 255.77000000000001)\n (154.96875, 4.2999999999999998, 228.63999999999999, 228.63999999999999, 220.69999999999999, 244.96000000000001, 247.75, 264.25999999999999, 255.77000000000001, 259.41000000000003, 18569.099999999999, 2413.0999999999999, 228.63999999999999, 220.69999999999999, 244.96000000000001, 247.75, 264.25999999999999, 255.7